# Conservation properties of delta function

In [1]:
import math
import numpy
import random

import pyibm

%matplotlib inline

In [2]:
pyibm.__version__

'0.1'

## Create Cartesian grids

In [3]:
# Set configuration parameters of computational grid.
config = dict(x=dict(start=-2.0, end=2.0, num_cells=20),
              y=dict(start=-2.0, end=2.0, num_cells=200))
# Create staggered grids.
grid = pyibm.GridBase(config=config)  # vertex grid
gridc = pyibm.GridCellCentered(grid=grid)  # cell-centered grid
gridx = pyibm.GridFaceX(grid=grid)  # x-face centered grid
gridy = pyibm.GridFaceY(grid=grid)  # y-face centered grid
print(gridc)
print(gridx)
print(gridy)

Grid(size=4000, shape=(200, 20), gridlines=[
Gridline(start=-2.0, end=2.0, size=20),
Gridline(start=-2.0, end=2.0, size=200)])
Grid(size=3800, shape=(200, 19), gridlines=[
Gridline(start=-2.0, end=2.0, size=19),
Gridline(start=-2.0, end=2.0, size=200)])
Grid(size=3980, shape=(199, 20), gridlines=[
Gridline(start=-2.0, end=2.0, size=20),
Gridline(start=-2.0, end=2.0, size=199)])


## Create immersed boundary

In [4]:
def circle(radius=0.5, center=(0.0, 0.0), ds=0.1, phi=0.0):
    """Compute uniformly distributed coordinates on circle."""
    xc, yc = center
    N = math.ceil(2 * math.pi * radius / ds)
    theta = numpy.linspace(0.0, 2 * math.pi, num=N + 1)[:-1] + phi
    x = xc + radius * numpy.cos(theta)
    y = yc + radius * numpy.sin(theta)
    return x, y

In [5]:
# Create immersed boundary.
ds = gridc.x.widths[0]
body = pyibm.Body(*circle(ds=ds), grid=gridc)
body

Body(ndim=2, size=16)

## Assemble delta operator

In [6]:
# Assemble delta kernel operator.
Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                          kernel=pyibm.delta_roma_et_al_1999,
                          kernel_size=2)
pyibm.print_matrix_info(Op, name='Delta')

# Check that operator remains the same if we increase
# the kernel window (additional values should be zero).
Op2 = pyibm.assemble_delta(body, gridc, gridx, gridy,
                           kernel=pyibm.delta_roma_et_al_1999,
                           kernel_size=3)
pyibm.print_matrix_info(Op2, name='Delta (bigger window)')

assert Op.nnz == Op2.nnz

Name:  Delta
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (32, 7780)
Size:  268
Min/Max:  -9.251858538542977e-15 111.11111111111117
Name:  Delta (bigger window)
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (32, 7780)
Size:  268
Min/Max:  -9.251858538542977e-15 111.11111111111117


## Assemble interpolation and spreading operators

In [7]:
# Assemble interpolation operator.
R = pyibm.assemble_R(gridx, gridy)
MHat = pyibm.assemble_MHat(gridx, gridy)
E = Op @ R @ MHat
pyibm.print_matrix_info(E, name='E')

Name:  E
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (32, 7780)
Size:  268
Min/Max:  -3.700743415417193e-17 0.444444444444445


In [8]:
# Assemble spreading operator (using surface areas).
S = pyibm.assemble_surfaces(body)
H = Op.T @ S
pyibm.print_matrix_info(H, name='H')

Name:  H
Type:  <class 'scipy.sparse.csc.csc_matrix'>
Shape:  (7780, 32)
Size:  268
Min/Max:  -1.8049480615320154e-15 21.676702446236487


## Check conservation properties

In [9]:
# Create array for Lagrangian forces.
Fx = numpy.random.rand(body.size)
Fy = numpy.random.rand(body.size)
F = numpy.empty(body.ndim * body.size)
F[::body.ndim], F[1::body.ndim] = Fx, Fy

# Spread Lagrangian forces to Eulerian grid.
f = H @ F
fx = f[:gridx.size].reshape(gridx.shape)
fy = f[gridx.size:].reshape(gridy.shape)

# Get surface area and cell volume.
dS = body.ds
dx = gridc.x.widths[gridc.ijk(body.neighbors[0])[0]]
dy = gridc.y.widths[gridc.ijk(body.neighbors[0])[1]]
dV = dx * dy

# Integrate forces.
print(numpy.sum(Fx) * dS, numpy.sum(Fy) * dS)
print(numpy.sum(fx) * dV, numpy.sum(fy) * dV)

1.5430309768233696 2.0255990412597487
1.5430309768233683 2.025599041259746


$$
\sum_{i, j, k} \mathbf{f}\left( \mathbf{x}_{i, j, k} \right) \Delta v_{i, j, k} = \sum_{m} \mathbf{F}\left( \mathbf{X}_{m} \right) \Delta V_{m}
$$

In [10]:
Xx, Yx = numpy.meshgrid(gridx.x.vertices, gridx.y.vertices)
Xy, Yy = numpy.meshgrid(gridy.x.vertices, gridy.y.vertices)

print((numpy.sum(Xy * fy) - numpy.sum(Yx * fx)) * dV)
print(numpy.sum(body.x * Fy - body.y * Fx) * dS)

-0.018795888190089934
-0.01879588819009014


$$
\sum_{i j, k} \mathbf{x}_{i j, k} \times \mathbf{f}\left( \mathbf{x}_{i, j, k} \right) \Delta v_{i, j, k} = \sum_{m} \mathbf{X}_{m} \times \mathbf{F}\left( \mathbf{X}_{m} \right) \Delta V_{m}
$$

In [11]:
# Set regularized delta kernel.
delta_kernel = pyibm.delta_roma_et_al_1999
vdelta_kernel = numpy.vectorize(delta_kernel)

# Create 1D domain.
x = numpy.linspace(0.0, 1.0, num=100)
dx = x[1] - x[0]

n_repeat = 200
tol = 1e-14
for _ in range(n_repeat):
    # Randomly define location of Lagrangian marker.
    xb = random.uniform(x[0] + 3 * dx, x[-1] - 3 * dx)
    # Compute.
    dist = numpy.abs(x - xb)
    res = dx * numpy.sum(vdelta_kernel(dist, dx))
    ans = 1.0
    assert abs(res - ans) < tol
    
    res = dx * numpy.sum((x - xb) * vdelta_kernel(dist, dx))
    ans = 0.0
    assert abs(res - ans) < tol

$$
\sum_{i j, k} \delta_{h}\left( \mathbf{X}_{i, j, k} - \mathbf{X} \right) \Delta v_{i, j, k} = 1
$$

$$
\sum_{i j, k}\left( \mathbf{x}_{i, j, k} - \mathbf{X} \right) \delta_{h}\left( \mathbf{x}_{i j, k} - \mathbf{X} \right) \Delta v_{i, j, k} = 0
$$